# Neuron set examples

In [2]:
import obi_one as obi
from pathlib import Path

### __Initialization:__ Loading a circuit

In [3]:
circuit_name = "N_10__top_nodes_dim6"

circuit_path_prefix = Path("../data/tiny_circuits")
matrix_path_prefix = Path("../data/connectivity_matrices")  # OPTIONAL: Connectivity matrix path only required for some of the node sets in this example notebook; can be set to None

circuit_path = circuit_path_prefix / circuit_name / "circuit_config.json"
if matrix_path_prefix is None:
    matrix_path = None
else:
    matrix_path = matrix_path_prefix / circuit_name / "connectivity_matrix.h5"
circuit = obi.Circuit(name=circuit_name, path=str(circuit_path), matrix_path=str(matrix_path))

print(f"Circuit '{circuit}' with {circuit.sonata_circuit.nodes[circuit.default_population_name].size} neurons and {circuit.sonata_circuit.edges[circuit.default_edge_population_name].size} synapses")
print(f"Default node population: '{circuit.default_population_name}'")
print(f"Default edge population: '{circuit.default_edge_population_name}'")

Circuit 'N_10__top_nodes_dim6' with 10 neurons and 176 synapses
Default node population: 'S1nonbarrel_neurons'
Default edge population: 'S1nonbarrel_neurons__S1nonbarrel_neurons__chemical'


### __Example 1:__ Adding node set dict to an existing SONATA circuit object + writing new node set .json file

In [ ]:
# Get SONATA circuit object
c = circuit.sonata_circuit
print("..." + str(c.node_sets.content)[-25:])

# Adding a node set to the circuit
obi.add_node_set_to_circuit(c, {"Layer23": {"layer": ["2", "3"]}})
print("..." + str(c.node_sets.content)[-55:])

# Adding a node set with an exising name => NOT POSSIBLE
# obi.add_node_set_to_circuit(c, {"Layer23": {"layer": ["2", "3"]}})  # AssertionError: Node set 'Layer23' already exists!

# Update/overwrite an existing node set
obi.add_node_set_to_circuit(c, {"Layer23": ["Layer2", "Layer3"]}, overwrite_if_exists=True)  # Update/overwrite
print("..." + str(c.node_sets.content)[-58:])

# Adding multiple node sets
obi.add_node_set_to_circuit(c, {"Layer45": ["Layer4", "Layer5"], "Layer56": ["Layer5", "Layer6"]})
print("..." + str(c.node_sets.content)[-124:])

# Add node set from NeuronSet object, resolved in circuit's default node population
neuron_set = obi.CombinedNeuronSet(node_sets=("Layer1", "Layer2", "Layer3"))
obi.add_node_set_to_circuit(c, {"Layer123": neuron_set.get_node_set_definition(circuit, circuit.default_population_name)})
print("..." + str(c.node_sets.content)[-168:])

# Adding a node sets based on previously added node sets
obi.add_node_set_to_circuit(c, {"AllLayers": ["Layer123", "Layer4", "Layer56"]})
print("..." + str(c.node_sets.content)[-216:])

# Write new circuit's node set file
obi.write_circuit_node_set_file(c, output_path="./", file_name="new_node_sets.json", overwrite_if_exists=True)

...id': [], 'layer': ['6']}}
... [], 'layer': ['6']}, 'Layer23': {'layer': ['2', '3']}}
...id': [], 'layer': ['6']}, 'Layer23': ['Layer2', 'Layer3']}
...id': [], 'layer': ['6']}, 'Layer23': ['Layer2', 'Layer3'], 'Layer45': ['Layer4', 'Layer5'], 'Layer56': ['Layer5', 'Layer6']}
...id': [], 'layer': ['6']}, 'Layer23': ['Layer2', 'Layer3'], 'Layer45': ['Layer4', 'Layer5'], 'Layer56': ['Layer5', 'Layer6'], 'Layer123': ['Layer1', 'Layer2', 'Layer3']}
...id': [], 'layer': ['6']}, 'Layer23': ['Layer2', 'Layer3'], 'Layer45': ['Layer4', 'Layer5'], 'Layer56': ['Layer5', 'Layer6'], 'Layer123': ['Layer1', 'Layer2', 'Layer3'], 'AllLayers': ['Layer123', 'Layer4', 'Layer56']}


### __Example 2:__ Use of different NeuronSet types

<u>Important</u>: In general, the validity of neuron set definitions is not checked during initialization, but when resolved within a specific circuit's node population

#### (a) __ExistingNeuronSet__, wrapper for an existing node set
<u>Note:</u> This neuron set does not resolve to a dict, since the underlying node set is already existing by definition

In [5]:
neuron_set = obi.PredefinedNeuronSet(node_set="Layer6", sample_percentage=100)
neuron_ids = neuron_set.get_neuron_ids(circuit, population=circuit.default_population_name)
print(f"{neuron_set.__class__.__name__} resolved in population '{circuit.default_population_name}' of circuit '{circuit}':")
print(f"> Neuron IDs ({len(neuron_ids)}): {neuron_ids}")
print(f"> Node set dict: {neuron_set.get_node_set_definition(circuit, circuit.default_population_name)}")

PredefinedNeuronSet resolved in population 'S1nonbarrel_neurons' of circuit 'N_10__top_nodes_dim6':
> Neuron IDs (9): [1 2 3 4 5 6 7 8 9]
> Node set dict: ['Layer6']


#### (b) __ExistingNeuronSet__, with random sub-sampling
<u>Note</u>: `sample_percentage` can be an absolute number or fraction

<u>Note 2</u>: Random sub-sampling will enforce resolving into a new node set

In [6]:
neuron_set = obi.PredefinedNeuronSet(node_set="Layer6", sample_percentage=50, sample_seed=1)
neuron_ids = neuron_set.get_neuron_ids(circuit, population=circuit.default_population_name)
print(f"{neuron_set.__class__.__name__} resolved in population '{circuit.default_population_name}' of circuit '{circuit}':")
print(f"> Neuron IDs ({len(neuron_ids)}): {neuron_ids}")
print(f"> Node set dict: {neuron_set.get_node_set_definition(circuit, circuit.default_population_name)}")

PredefinedNeuronSet resolved in population 'S1nonbarrel_neurons' of circuit 'N_10__top_nodes_dim6':
> Neuron IDs (4): [2 3 5 9]
> Node set dict: {'population': 'S1nonbarrel_neurons', 'node_id': [2, 3, 5, 9]}


#### (c) __CombinedNeuronSet__, based on combining existing (named) node sets

In [7]:
neuron_set = obi.CombinedNeuronSet(circuit=circuit, node_sets=("L6_BPC", "L6_TPC:A"), sample_percentage=100)
neuron_ids = neuron_set.get_neuron_ids(circuit, population=circuit.default_population_name)
print(f"{neuron_set.__class__.__name__} resolved in population '{circuit.default_population_name}' of circuit '{circuit}':")
print(f"> Neuron IDs ({len(neuron_ids)}): {neuron_ids}")
print(f"> Node set dict: {neuron_set.get_node_set_definition(circuit, circuit.default_population_name)}")

CombinedNeuronSet resolved in population 'S1nonbarrel_neurons' of circuit 'N_10__top_nodes_dim6':
> Neuron IDs (6): [1 2 6 7 8 9]
> Node set dict: ['L6_BPC', 'L6_TPC:A']


#### (d) __CombinedNeuronSet__, based on combining existing (named) node sets, with random sub-sampling
<u>Note</u>: `sample_percentage` can be an absolute number or fraction

In [8]:
neuron_set = obi.CombinedNeuronSet(node_sets=("L6_BPC", "L6_TPC:A"), sample_percentage=50, sample_seed=0)
neuron_ids = neuron_set.get_neuron_ids(circuit, circuit.default_population_name)
print(f"{neuron_set.__class__.__name__} resolved in population '{circuit.default_population_name}' of circuit '{circuit}':")
print(f"> Neuron IDs ({len(neuron_ids)}): {neuron_ids}")
print(f"> Node set dict: {neuron_set.get_node_set_definition(circuit, circuit.default_population_name)}")

CombinedNeuronSet resolved in population 'S1nonbarrel_neurons' of circuit 'N_10__top_nodes_dim6':
> Neuron IDs (3): [2 8 9]
> Node set dict: {'population': 'S1nonbarrel_neurons', 'node_id': [2, 8, 9]}


#### (e) __IDNeuronSet__, based on individual neuron IDs

In [9]:
neuron_set = obi.IDNeuronSet(neuron_ids=obi.NamedTuple(name="IDNeuronSet1", elements=(0, 2, 8)))
neuron_ids = neuron_set.get_neuron_ids(circuit, circuit.default_population_name)
print(f"{neuron_set.__class__.__name__} resolved in population '{circuit.default_population_name}' of circuit '{circuit}':")
print(f"> Neuron IDs ({len(neuron_ids)}): {neuron_ids}")
print(f"> Node set dict: {neuron_set.get_node_set_definition(circuit, circuit.default_population_name)}")

IDNeuronSet resolved in population 'S1nonbarrel_neurons' of circuit 'N_10__top_nodes_dim6':
> Neuron IDs (3): [0 2 8]
> Node set dict: {'population': 'S1nonbarrel_neurons', 'node_id': [0, 2, 8]}


#### (f) __IDNeuronSet__, based on individual neuron IDs, with random sub-sampling
<u>Note</u>: `sample_percentage` can be an absolute number or fraction

In [10]:
neuron_set = obi.IDNeuronSet(neuron_ids=obi.NamedTuple(name="IDNeuronSet1", elements=range(10)), sample_percentage=50, sample_seed=999)
neuron_ids = neuron_set.get_neuron_ids(circuit, circuit.default_population_name)
print(f"{neuron_set.__class__.__name__} resolved in population '{circuit.default_population_name}' of circuit '{circuit}':")
print(f"> Neuron IDs ({len(neuron_ids)}): {neuron_ids}")
print(f"> Node set dict: {neuron_set.get_node_set_definition(circuit, circuit.default_population_name)}")

IDNeuronSet resolved in population 'S1nonbarrel_neurons' of circuit 'N_10__top_nodes_dim6':
> Neuron IDs (5): [1 4 5 6 8]
> Node set dict: {'population': 'S1nonbarrel_neurons', 'node_id': [1, 4, 5, 6, 8]}


#### (g) __PropertyNeuronSet__, based on neuron properties
<u>Note</u>: Optionally, instead of keeping the synbolic notation, neuron IDs can be resolved to individual IDs by `force_resolve_ids=True`.

In [11]:
neuron_set = obi.PropertyNeuronSet(
    property_filter=obi.NeuronPropertyFilter(filter_dict={"layer": ["5", "6"], "synapse_class": ["EXC"]}),
)
neuron_ids = neuron_set.get_neuron_ids(circuit, circuit.default_population_name)
print(f"{neuron_set.__class__.__name__} resolved in population '{circuit.default_population_name}' of circuit '{circuit}':")
print(f"> Neuron IDs ({len(neuron_ids)}): {neuron_ids}")
print(f"> Node set dict: {neuron_set.get_node_set_definition(circuit, circuit.default_population_name)}")

# # Optional: Individual neuron IDs resolved
print(f"> Node set dict with IDs resolved [OPTIONAL]: {neuron_set.get_node_set_definition(circuit, circuit.default_population_name, force_resolve_ids=True)}")

PropertyNeuronSet resolved in population 'S1nonbarrel_neurons' of circuit 'N_10__top_nodes_dim6':
> Neuron IDs (9): [1 2 3 4 5 6 7 8 9]
> Node set dict: {'layer': ['5', '6'], 'synapse_class': 'EXC'}
> Node set dict with IDs resolved [OPTIONAL]: {'population': 'S1nonbarrel_neurons', 'node_id': [1, 2, 3, 4, 5, 6, 7, 8, 9]}


#### (h) __PropertyNeuronSet__, based on neuron properties, combined with exising (named) node sets
<u>Note</u>: In this case, individual neuron IDs will always be resolved since a combination of properties and node sets is not possible in SONATA node sets otherwise!

In [12]:
neuron_set = obi.PropertyNeuronSet(
    property_filter=obi.NeuronPropertyFilter(filter_dict={"synapse_class": ["INH"]}),
    node_sets=("Layer2", "Layer3")
)
neuron_ids = neuron_set.get_neuron_ids(circuit, circuit.default_population_name)
print(f"{neuron_set.__class__.__name__} resolved in population '{circuit.default_population_name}' of circuit '{circuit}':")
print(f"> Neuron IDs ({len(neuron_ids)}): {neuron_ids}")
print(f"> Node set dict: {neuron_set.get_node_set_definition(circuit, circuit.default_population_name)}")

PropertyNeuronSet resolved in population 'S1nonbarrel_neurons' of circuit 'N_10__top_nodes_dim6':
> Neuron IDs (1): [0]
> Node set dict: {'population': 'S1nonbarrel_neurons', 'node_id': [0]}


#### (i) __VolumetricCountNeuronSet__, sample a spatial neighborhood


In [13]:
neuron_set = obi.VolumetricCountNeuronSet(
    ox=10.0,
    oy=25.0,
    oz=100.0,
    n=5,
    property_filter=obi.NeuronPropertyFilter(filter_dict={"layer": ["5", "6"], "synapse_class": ["EXC"]}),
)

neuron_ids = neuron_set.get_neuron_ids(circuit, circuit.default_population_name)
print(f"{neuron_set.__class__.__name__} resolved in population '{circuit.default_population_name}' of circuit '{circuit}':")
print(f"> Neuron IDs ({len(neuron_ids)}): {neuron_ids}")
print(f"> Node set dict: {neuron_set.get_node_set_definition(circuit, circuit.default_population_name)}")

print("Printing neuron locations to prove spatial compactness:")
circuit.sonata_circuit.nodes[circuit.default_population_name].get(neuron_ids, properties=["x", "y", "z"])

VolumetricCountNeuronSet resolved in population 'S1nonbarrel_neurons' of circuit 'N_10__top_nodes_dim6':
> Neuron IDs (5): [3 5 6 7 9]
> Node set dict: {'population': 'S1nonbarrel_neurons', 'node_id': [9, 6, 7, 5, 3]}
Printing neuron locations to prove spatial compactness:


,x,y,z
node_ids,,,
3,3627.362786,-1087.181103,-2897.810749
5,3592.676277,-1025.078101,-2898.148159
6,3639.411619,-1055.078853,-2834.399699
7,3624.201128,-1018.365265,-2890.523732
9,3663.108874,-1035.551752,-2810.042016


#### (j) __VolmetricRadiusNeuronSet__, also neigborhood, but fixed radius
<u>Note</u>: Can be combined with random subsampling

In [14]:
neuron_set = obi.VolumetricRadiusNeuronSet(
    ox=10.0,
    oy=25.0,
    oz=100.0,
    radius=150.0,
    property_filter=obi.NeuronPropertyFilter(filter_dict={"layer": ["5", "6"], "synapse_class": ["EXC"]}),
    sample_percentage=50
)

neuron_ids = neuron_set.get_neuron_ids(circuit, circuit.default_population_name)
print(f"{neuron_set.__class__.__name__} resolved in population '{circuit.default_population_name}' of circuit '{circuit}':")
print(f"> Neuron IDs ({len(neuron_ids)}): {neuron_ids}")
print(f"> Node set dict: {neuron_set.get_node_set_definition(circuit, circuit.default_population_name)}")

print("Printing neuron locations to prove spatial compactness:")
circuit.sonata_circuit.nodes[circuit.default_population_name].get(neuron_ids, properties=["x", "y", "z"])

VolumetricRadiusNeuronSet resolved in population 'S1nonbarrel_neurons' of circuit 'N_10__top_nodes_dim6':
> Neuron IDs (4): [3 4 6 9]
> Node set dict: {'population': 'S1nonbarrel_neurons', 'node_id': [3, 4, 6, 9]}
Printing neuron locations to prove spatial compactness:


,x,y,z
node_ids,,,
3,3627.362786,-1087.181103,-2897.810749
4,3537.776871,-1029.445880,-2890.783793
6,3639.411619,-1055.078853,-2834.399699
9,3663.108874,-1035.551752,-2810.042016


#### (k) __SimplexMembershipBasedNeuronSet__, choose neurons based on their membership in simplices with a given source or target neuron
<u>Note</u>: Needs `connalysis` to run and needs a circuit with a connectivity matrix!

In [15]:
from connalysis.network import topology # Needs to be installed to run this example

In [16]:
assert matrix_path is not None, "ERROR: Circuit with connectivity matrix required!"
neuron_set = obi.SimplexMembershipBasedNeuronSet(
    central_neuron_id = 9,
    dim = 2,
    central_neuron_simplex_position = 'source',
    subsample = True ,
    n_count_max = 3,
    subsample_method = 'node_participation', #"random" is another option,
    property_filter=obi.NeuronPropertyFilter( ),
)

neuron_ids = neuron_set.get_neuron_ids(circuit, circuit.default_population_name)
print(f"{neuron_set.__class__.__name__} resolved in population '{circuit.default_population_name}' of circuit '{circuit}':")
print(f"\n> Neuron IDs ({len(neuron_ids)}): {neuron_ids}\n")
print(f"\n> Node set dict: {neuron_set.get_node_set_definition(circuit, circuit.default_population_name)}")

print("\n> Node selection\n")
sub_conn = circuit.connectivity_matrix.subpopulation(neuron_ids)
display(sub_conn.vertices)
print(f"> Simplex counts in subcircuit:\n {topology.simplex_counts(sub_conn.matrix)}\n")

[2025-09-08 17:40:06,950] INFO: COMPUTE list of simplices by dimension
SimplexMembershipBasedNeuronSet resolved in population 'S1nonbarrel_neurons' of circuit 'N_10__top_nodes_dim6':

> Neuron IDs (3): [7 8 9]

[2025-09-08 17:40:06,976] INFO: COMPUTE list of simplices by dimension

> Node set dict: {'population': 'S1nonbarrel_neurons', 'node_id': [9, 7, 8]}

> Node selection



,node_ids,etype,layer,mtype,synapse_class,x,y,z
0,7,cADpyr,6,L6_TPC:A,EXC,3624.201128,-1018.365265,-2890.523732
1,8,cADpyr,6,L6_TPC:A,EXC,3547.925890,-1083.981005,-2933.071149
2,9,cADpyr,6,L6_TPC:A,EXC,3663.108874,-1035.551752,-2810.042016


> Simplex counts in subcircuit:
 dim
0    3
1    3
2    1
Name: simplex_count, dtype: int64



#### (l) __SimplexNeuronSet__, choose neurons that form simplices of a given dimension with a chosen source or target neuron
<u>Note</u>: Needs `connalysis` to run and needs a circuit with a connectivity matrix!

In [17]:
assert matrix_path is not None, "ERROR: Circuit with connectivity matrix required!"
neuron_set = obi.SimplexNeuronSet(
    central_neuron_id = 9, 
    dim = 100,
    central_neuron_simplex_position = 'source',
    subsample = True ,
    n_count_max = 3,
    property_filter=obi.NeuronPropertyFilter( filter_dict={"synapse_class": ["EXC"]}),
)

neuron_ids = neuron_set.get_neuron_ids(circuit, circuit.default_population_name)
print(f"{neuron_set.__class__.__name__} resolved in population '{circuit.default_population_name}' of circuit '{circuit}':")
print(f"\n> Neuron IDs ({len(neuron_ids)}): {neuron_ids}\n")
print(f"\n> Node set dict: {neuron_set.get_node_set_definition(circuit, circuit.default_population_name)}")

print("\n> Node selection\n")
sub_conn = circuit.connectivity_matrix.subpopulation(neuron_ids)
display(sub_conn.vertices)
print(f"> Simplex counts in subcircuit:\n {topology.simplex_counts(sub_conn.matrix)}\n")


[2025-09-08 17:40:08,190] INFO: COMPUTE list of simplices by dimension
[2025-09-08 17:40:08,191] INFO: > Dimension not attained using dimension 3 instead.
[2025-09-08 17:40:08,192] INFO: > n_count_max is too small to form a single 3-simplex, sampling n_count_max =                 4 neurons instead.
[2025-09-08 17:40:08,192] INFO: No subselection required
SimplexNeuronSet resolved in population 'S1nonbarrel_neurons' of circuit 'N_10__top_nodes_dim6':

> Neuron IDs (4): [4 7 8 9]

[2025-09-08 17:40:08,218] INFO: COMPUTE list of simplices by dimension
[2025-09-08 17:40:08,220] INFO: > Dimension not attained using dimension 3 instead.
[2025-09-08 17:40:08,220] INFO: > n_count_max is too small to form a single 3-simplex, sampling n_count_max =                 4 neurons instead.
[2025-09-08 17:40:08,220] INFO: No subselection required

> Node set dict: {'population': 'S1nonbarrel_neurons', 'node_id': [4, 7, 8, 9]}

> Node selection



,node_ids,etype,layer,mtype,synapse_class,x,y,z
0,4,cADpyr,6,L6_IPC,EXC,3537.776871,-1029.445880,-2890.783793
1,7,cADpyr,6,L6_TPC:A,EXC,3624.201128,-1018.365265,-2890.523732
2,8,cADpyr,6,L6_TPC:A,EXC,3547.925890,-1083.981005,-2933.071149
3,9,cADpyr,6,L6_TPC:A,EXC,3663.108874,-1035.551752,-2810.042016


> Simplex counts in subcircuit:
 dim
0    4
1    7
2    4
3    1
Name: simplex_count, dtype: int64



### __Example 3:__ Writing a NeuronSet to a SONATA node set file
<u>Note</u>: A NeuronSet name must be set, which will be the name of the SONATA node set. The name must not exist!

<u>Note 2</u>: The node sets file name is by default taken from the original circuit. An alternative name can optionally be provided.

<u>Note 3</u>: Overwrite (`overwrite_if_exists`) and append (`append_if_exists`) options exist.

In [18]:
output_path = "./"

# Write new file, overwrite if existing
neuron_set = obi.CombinedNeuronSet(node_sets=("Layer1", "Layer2", "Layer3"))
nset_file = neuron_set.to_node_set_file(circuit, circuit.default_population_name, output_path=output_path, overwrite_if_exists=True, optional_node_set_name="L123")

# Append to existing file, but name already exists => NOT POSSIBLE
# nset_file = neuron_set.to_node_set_file(circuit, circuit.default_population_name, output_path=output_path, append_if_exists=True, optional_node_set_name="L123")  # AssertionError: Appending not possible, node set 'L123' already exists!

# Append to existing file
neuron_set = obi.CombinedNeuronSet(node_sets=("Layer4", "Layer5", "Layer6"))
nset_file = neuron_set.to_node_set_file(circuit, circuit.default_population_name, output_path=output_path, append_if_exists=True, optional_node_set_name="L456")

if Path(nset_file).exists():
    print(f"Node set file: {nset_file}")

Node set file: node_sets.json
